# Fine Tuning ResNet50 for ICB

In [1]:
import numpy as np
import tensorflow as tf
from matplotlib import pyplot as plt
from tensorflow import keras as kr

%matplotlib inline

In [ ]:
# Implementation Dependant Globals
REQUIRED_DIMENSIONS = (300, 300)
TRAIN_BATCH_SIZE  = 10
VALIDATION_BATCH_SIZE = 10
TEST_BATCH_SIZE = 10

# Import dataset
%run DataAugmentation.ipynb 

#### Fetch & Download ResNet50

In [3]:
# Fetch ResNet50 implementation from keras
resnet50 = kr.applications.resnet50
ResNet50 = resnet50.ResNet50

resnet_model = ResNet50(
    weights='imagenet', 
    include_top=False, 
    input_shape= REQUIRED_DIMENSIONS + (3,)
)

#### Build Fine Tuned Model

In [7]:
# Ease of Access
Model, Dropout, Flatten, Dense = kr.models.Model, kr.layers.Dropout, kr.layers.Flatten, kr.layers.Dense

CLASSES = ["biofuel", "non_biofuel"]
FC_LAYERS = [1024, 1024]
dropout = 0.5

for layer in resnet_model.layers:
    layer.trainable = False

output = resnet_model.output
output = Flatten()(output)

for fc in FC_LAYERS:
    # New FC layer, random init
    output = Dense(fc, activation='relu')(output) 
    output = Dropout(dropout)(output)

# New softmax layer
predictions = Dense(len(CLASSES), activation='softmax')(output) 

# Finally join everything
finetuned_model = Model(inputs=resnet_model.input, outputs=predictions)
finetuned_model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 300, 300, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 306, 306, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 150, 150, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 150, 150, 64) 256         conv1_conv[0][0]                 
______________________________________________________________________________________________

#### Train Model

In [ ]:
finetune_model.compile(
    kr.optimser.Adam(lr=0.00001), 
    loss='binary_crossentropy', 
    metrics=['accuracy']
)

history = finetune_model.fit_generator(
    train_set, 
    epochs=10, 
    workers=8, 
    steps_per_epoch=train_set.samples/train_set.batch_size, 
    shuffle=True,
)